In [1]:
import os
import matplotlib.pyplot as plt

path='/kaggle/input/plantvillage29classes/train/'
plt.figure(figsize=(70,70))
count=0
plant_names=[]
total_images=0

for i in os.listdir(path):
  count+=1
  plant_names.append(i)

  images_path=[img for img in os.listdir(path+"/"+i) if img.endswith(('.png', '.jpg', '.jpeg'))]
  print("Number of images of "+i+":",len(images_path),"||",end=" ")
  total_images+=len(images_path)

  if len(images_path) > 0:
    image_show=plt.imread(path+"/"+i+"/"+images_path[0])
    #plt.imshow(image_show)
    #plt.xlabel(i)
  
  #plt.xticks([])
  #plt.yticks([])

print("Total number of images we have",total_images)


Number of images of labels: 0 || Number of images of images: 2017 || Total number of images we have 2017


<Figure size 7000x7000 with 0 Axes>

In [2]:
print(plant_names)
print(len(plant_names))

['labels', 'images']
2


In [ ]:
import tensorflow
from tensorflow import keras
from keras.models import Sequential, load_model, Model
from keras.layers import Conv2D, MaxPool2D, AveragePooling2D, Dense, Flatten, ZeroPadding2D, BatchNormalization, Activation, Add, Input, Dropout, GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.initializers import glorot_uniform
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [3]:
#CLAHE Preprocessing
import cv2
import os

# Dataset directoryD:\Miniproject\New Plant Diseases Dataset(Augmented)\New Plant Diseases Dataset(Augmented)
dataset_dir = '/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'

# Output directory
output_dir = "uma_preprocessed"

# Create output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Define CLAHE parameters (optional)
clipLimit = 5

# Function to apply CLAHE preprocessing
def apply_clahe(image):
    # Split the image into channels
    b, g, r = cv2.split(image)

    # Apply CLAHE to each channel
    clahe = cv2.createCLAHE(clipLimit=clipLimit)
    clahe_b = clahe.apply(b)
    clahe_g = clahe.apply(g)
    clahe_r = clahe.apply(r)

    # Merge the channels back together
    processed_image = cv2.merge([clahe_b, clahe_g, clahe_r])

    return processed_image

# Iterate through class subfolders
for class_name in os.listdir(dataset_dir):
    if os.path.isdir(os.path.join(dataset_dir, class_name)):
        # Create output subfolder for this class
        output_class_dir = os.path.join(output_dir, class_name)
        if not os.path.exists(output_class_dir):
            os.makedirs(output_class_dir)

        # Iterate through images in the class subfolder
        for filename in os.listdir(os.path.join(dataset_dir, class_name)):
            image_path = os.path.join(dataset_dir, class_name, filename)
            image = cv2.imread(image_path)

            # Preprocess the image (optional)
            if apply_clahe:
                processed_image = apply_clahe(image)
            else:
                processed_image = image

            # Save the preprocessed image
            output_path = os.path.join(output_class_dir, filename)
            cv2.imwrite(output_path, processed_image)

print("Preprocessing complete.")

Preprocessing complete.


In [ ]:
!conda install -y gdown 

In [ ]:
!

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

dataset_path = '/kaggle/working/uma_preprocessed'

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Load the entire dataset
full_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(full_dataset))  # 80% for training
val_size = len(full_dataset) - train_size  # 20% for validation
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

# Create data loaders for the training and validation sets
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)

In [9]:
# --- Imports --- #
import torch
import torch.nn as nn
import torch.nn.functional as F

# --- Channel Attention (CA) Layer --- #
class CALayer(nn.Module):
    def __init__(self, channel, reduction=4):
        super(CALayer, self).__init__()
        # global average pooling: feature --> point
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        # feature channel downscale and upscale --> channel weight
        self.conv_du = nn.Sequential(
                nn.Conv2d(channel, channel // reduction, 1, padding=0, bias=True),
                nn.ReLU(inplace=True),
                nn.Conv2d(channel // reduction, channel, 1, padding=0, bias=True),
                nn.Sigmoid()
        )

    def forward(self, x):
        y = self.avg_pool(x)
        y = self.conv_du(y)
        return x * y 

class PCFAN(nn.Module):
    def __init__(self):
        super(PCFAN, self).__init__()
        self.layer1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1)
            )
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1)
            )
        self.layer5 = nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1)
        self.layer6 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1)
            )
        self.layer7 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1)
            )
        self.layer9 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)
        self.layer10 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1)
            )
        self.layer11 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1)
            )
        self.Attention_1 = CALayer(channel=32)
        self.Attention_2 = CALayer(channel=64)
        self.Attention_3 = CALayer(channel=128)
        self.Attention_4 = CALayer(channel=192)
        self.final = nn.Conv2d(224,3,kernel_size=3,padding=1)
        self.tanh = nn.Tanh()

    def three_scale_feature_extraction_phase(self, picture):
        B,C,H,W = picture.size()
        picture0 = picture
        picture = self.layer1(picture)
        picture = self.layer2(picture) + picture
        map1 = self.layer3(picture) + picture     

        map2 = self.layer5(map1)
        map2 = self.layer6(map2) + map2
        map2 = self.layer7(map2) + map2 
        map2 = nn.UpsamplingBilinear2d((H//2,W//2))(map2)

        map3 = self.layer9(map2)    
        map3 = self.layer10(map3) + map3
        map3 = self.layer11(map3) + map3 

        map1 = self.Attention_1(map1)
        map2 = self.Attention_2(map2)
        map3 = self.Attention_3(map3)
        map3 = nn.UpsamplingBilinear2d((H//2,W//2))(map3)
        map23 = torch.cat((map2,map3),1)
        map23 = self.Attention_4(map23)
        map23 = nn.UpsamplingBilinear2d((H,W))(map23)

        map123 = torch.cat((map23,map1),1)
        out = self.tanh(self.final(map123))
        out = torch.add(picture0,out)
        return out

    def extract_features(self, picture):
        features = self.three_scale_feature_extraction_phase(picture)
        return features


In [ ]:
import os
import pickle

# Instantiate the model
model = PCFAN()

# Define the directory for the extracted features
extracted_features_dir = '/kaggle/working/extracted_features'

# Loop over the dataset
for i, data in enumerate(train_loader, 0):
    # Get the inputs; data is a list of [inputs, labels]
    inputs, labels = data

    # Extract feature maps
    feature_maps = model.extract_features(inputs)

    # Get the class name from the label
    class_name = train_loader.dataset.dataset.classes[labels[0].item()]

    # Create a directory for this class, if it doesn't exist already
    class_dir = os.path.join(extracted_features_dir, class_name)
    os.makedirs(class_dir, exist_ok=True)

    # Save the feature maps to a pickle file in this directory
    with open(os.path.join(class_dir, f'feature_maps_{i}.pkl'), 'wb') as f:
        pickle.dump(feature_maps, f)


In [40]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class SingleFolderDataset(Dataset):
    def __init__(self, folder, transform=None):
        self.transform = transform
        self.image_files = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith('.png') or f.endswith('.jpg') or f.endswith('.JPG')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        img = Image.open(img_path)
        if self.transform:
            img = self.transform(img)
        return img

dataset_path = '/kaggle/working/uma_preprocessed/Peach___healthy'

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Load the dataset
dataset = SingleFolderDataset(dataset_path, transform=transform)

# Create a data loader for the dataset
data_loader = DataLoader(dataset, batch_size=64, shuffle=True)

In [ ]:
import os
import pickle

# Instantiate the model
model = PCFAN()

# Define the directory for the extracted features
extracted_features_dir = '/kaggle/working/extracted_features'

# Create the directory if it does not exist
os.makedirs(extracted_features_dir, exist_ok=True)

# Loop over the dataset
for i, inputs in enumerate(data_loader, 0):
    # Extract feature maps
    feature_maps = model.extract_features(inputs)

    # Save the feature maps to a pickle file in the directory
    with open(os.path.join(extracted_features_dir, f'feature_maps_{i}.pkl'), 'wb') as f:
        pickle.dump(feature_maps, f)


In [ ]:
!zip -r file.zip /kaggle/working/extracted_features

  adding: kaggle/working/extracted_features/ (stored 0%)
  adding: kaggle/working/extracted_features/feature_maps_6.pkl (deflated 13%)
  adding: kaggle/working/extracted_features/feature_maps_17.pkl (deflated 11%)
  adding: kaggle/working/extracted_features/feature_maps_18.pkl (deflated 11%)
  adding: kaggle/working/extracted_features/feature_maps_7.pkl (deflated 13%)
  adding: kaggle/working/extracted_features/feature_maps_1.pkl (deflated 13%)
  adding: kaggle/working/extracted_features/feature_maps_23.pkl (deflated 10%)
  adding: kaggle/working/extracted_features/feature_maps_15.pkl (deflated 12%)
  adding: kaggle/working/extracted_features/feature_maps_26.pkl (deflated 12%)
  adding: kaggle/working/extracted_features/feature_maps_16.pkl (deflated 12%)
  adding: kaggle/working/extracted_features/feature_maps_22.pkl (deflated 14%)
  adding: kaggle/working/extracted_features/feature_maps_19.pkl (deflated 13%)
  adding: kaggle/working/extracted_features/feature_maps_5.pkl (deflated 12%)

In [ ]:
from IPython.display import FileLink
FileLink(r'file.zip')

In [38]:
!rm -rf /kaggle/working/extracted_features

In [39]:
!rm -rf /kaggle/working/file.zip